In [67]:
#from torchtext.data import Field, Dataset
import torch
from torch.utils.data import Dataset
from pytorch_transformers import BertTokenizer
from torchvision import transforms
import pandas as pd
from torch.nn import functional as F
import sys
import numpy as np
sys.path.append('../source/dataloaders/')

In [4]:
import data_format_utils as dfu

Using TensorFlow backend.


In [11]:
df = pd.read_pickle('../data/preprocessed/semcor_debug.pkl')

In [6]:
DEF_TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
from torchtext.data import Field
tokenize = lambda x: DEF_TOKENIZER(x)
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)

LABEL = Field(sequential=False, use_vocab=False)

In [63]:
row = df.iloc[0]
def find_token(_tokenized,_target):
    idxs_token  = [i for i,word  in \
                         enumerate(_tokenized) \
                         if word == _target.lower()]
    return idxs_token[0]

def get_index_of_sep(_token_sent):
        """
        Get index of sep token and generate sentence index array
        """ 
        _index_sep_tokens = [i for i,word  in enumerate(_token_sent) \
                           if word == '[SEP]']
        _sentence_indexes = np.array([0]*(_index_sep_tokens[0]+1)\
                                     +[1]*(_index_sep_tokens[1]-_index_sep_tokens[0]))
        return _sentence_indexes


def tranform_row(_row):
    sent = dfu.format_sentences_BERT(_row)
    tok_sent = DEF_TOKENIZER.tokenize(sent)
    target_word_tokens = DEF_TOKENIZER.tokenize(_row.target_word)
    idx = find_token(tok_sent,target_word_tokens[0])
    idx_sent = DEF_TOKENIZER.convert_tokens_to_ids(tok_sent)
    sentence_embed = get_index_of_sep(tok_sent)
    return idx_sent, sentence_embed, idx
    
    
    
out = tranform_row(row)
    

In [62]:
len(out[0])

46

In [126]:
MAX_LEN = 128

class CorpusTransformDataset(Dataset):
    """
    pytorch dataset handling class    
    """
    def __init__(self, data,pad_len=MAX_LEN):
        self.corpus_dataframe = data
        self.pad_len = pad_len

    def __len__(self):
        return self.corpus_dataframe.shape[0]

    def __getitem__(self, idx):
        raw_row = self.corpus_dataframe.iloc[idx]
        token_idxs_raw,sentence_embed_raw,target_token_idx = tranform_row(raw_row)
        token_idxs_raw = torch.tensor(token_idxs_raw)
        sentence_embed_raw = torch.tensor(sentence_embed_raw, dtype=torch.int64)

        if token_idxs_raw.shape[0] >= self.pad_len:
            token_idxs_pad = token_idxs_raw[:self.pad_len]
            sentence_embed_pad = sentence_embed_raw[:self.pad_len]
        else: # pad with 0s and 1s
            token_idxs_pad = F.pad(token_idxs_raw,(0,self.pad_len-token_idxs_raw.shape[0]))
            sentence_embed_pad = F.pad(sentence_embed_raw,(0,self.pad_len-sentence_embed_raw.shape[0]),value=1)
        
        return (token_idxs_pad,  # Input token encodings,
                sentence_embed_pad, # Sentence encoding
                torch.tensor(target_token_idx, dtype=torch.int64), # Target token indexes
                torch.tensor(raw_row['is_proper_gloss'],dtype=torch.int64)) # Labels

In [129]:
train_dataset = CorpusTransformDataset(df,pad_len=128)
train_dataset[0][0]

tensor([  101,  2006,  1035,  1996,  1035,  2060,  1035,  2192,  1037,  3129,
         2040,  2467,  2038,  2042, 21813,  1998, 20865,  3512,  2089,  3402,
         2468, 13135,  4851, 25428,  2360,  2005,  2128, 12054, 25863,  2008,
         2010,  2564,  2145,  4858,  2032, 16166,   102,  2191,  1037,  5456,
         1010,  2191,  1037,  2047,  4531,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [107]:
out = train_dataset[0]
out[0][:2]

tensor([ 101, 2006])

In [102]:
padd = F.pad(out[0],(0,128-out[0].shape[0]))

In [104]:
F.pad??

In [ ]:
ou

In [36]:
#train_dataset[0]
dfu.tokenize_and_index(df,output_len=128,weak_supervision=True,
                       tokenizer=DEF_TOKENIZER,verbose=False)

TypeError: tokenize_and_index() got an unexpected keyword argument 'verbose'

In [38]:
dfu.format_sentences_BERT

<function data_format_utils.format_sentences_BERT(_row, weak_supervision=False)>

In [28]:
F.pad

AttributeError: module 'torch.functional' has no attribute 'pad'